In [1]:
# this needs to be run for each new runtime
# because colab has scikit-learn 1.0.2 pre-installed
# and we need newer version (1.2.0 and higher)
# to use .set_output() method
!pip install scikit-learn --upgrade

# if you plan on running the whole notebook again during the same runtime
# you can comment the line above

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 69.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

## 1. Data reading

In [3]:
# Step 1: Extract Sheet ID and Sheet name from the link
link = "https://docs.google.com/spreadsheets/d/1rsoLi9aKL_JEldxuaIaI81JrW0X-y4EYzVMWBOcTWMs/edit#gid=2106468889"

sheet_id = link.split("/")[5]
sheet_name = link.split("gid=")[1]

# Step 2: Construct the URL
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={sheet_name}"

# Step 3: Read the URL as a pandas DataFrame
df = pd.read_csv(url)

# Print the DataFrame
df

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,Expensive,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,8450,65.0,856,3,0,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,9600,80.0,1262,3,1,0,2,298,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,11250,68.0,920,3,1,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,9550,60.0,756,3,1,0,3,0,0,0,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,14260,84.0,1145,4,1,0,3,192,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,62.0,953,3,1,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1456,13175,85.0,1542,3,2,0,2,349,0,0,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1457,9042,66.0,1152,4,2,0,1,0,0,1,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal
1458,9717,68.0,1078,2,0,0,1,366,0,0,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal


## 2. Data reading & splitting

In [4]:
# Removing irrelevant columns:

# Define the list of columns to be removed
columns_to_remove = ['Alley', 'Utilities', 'PoolQC', 'Fence', 'MiscFeature']

# Remove the columns from the dataframe
df = df.drop(columns_to_remove, axis=1)


In [6]:
# X and y creation:

# Assuming 'expensive' is the target variable
y = df.pop('Expensive')
X = df

In [7]:
# data splitting:

from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=88)


## 3. Categorical encoding - "Automated" approach (Using Pipelines)

### 3.1. Creating the "numeric pipe" and the "categoric pipe"

In [8]:
# select categorical and numerical column names
X_cat_columns = X.select_dtypes(exclude="number").copy().columns
X_num_columns = X.select_dtypes(include="number").copy().columns

In [9]:


# Numerical pipeline | Without Scaling (Current Approach):
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])


# Categorical pipeline
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='N_A')),
    ('ordinal_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
    ('onehot_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])


### 3.2. Using `ColumnTransformer` a pipeline with 2 branches (the `preprocessor`)

We simply tell the pipeline the following:

- One branch, called `"num_pipe"`, will apply the steps in the `numeric_pipe` to the columns named in `X_num_columns`
- The second branch, called `"cat_pipe"`, will apply the steps in the `categoric_pipe` to the columns named in `X_cat_columns`

In [10]:
from sklearn.compose import ColumnTransformer

# ColumnTransformer for preprocessing
preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, X_num_columns),
    ('cat_pipeline', cat_pipeline, X_cat_columns)
])


### 3.3. Creating the `full_pipeline` (`preprocessor` +  Random Forest Classifier)

Pipelines are modular. The `preprocessor` we created above with the `ColumnTransformer` can become now a step in a new pipeline, that we'll call `full_piepline` and will include, as a last step, a  Random Forest Classifier model:

In [20]:
import warnings
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

# Suppress FutureWarnings temporarily
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

    # Full pipeline with preprocessor and Random Forest classifier
    full_pipeline = make_pipeline(preprocessor, RandomForestClassifier())

    # Fit the pipeline on the training data
    full_pipeline.fit(X_train, y_train)


In [22]:
# Make predictions on the training data
pipeline_pred = full_pipeline.predict(X_train)
pipeline_pred

array([0, 1, 0, ..., 0, 0, 0])

In [23]:
# Calculate accuracy score
accuracy = accuracy_score(y_true=y_train, y_pred=pipeline_pred)
accuracy

1.0

In [24]:
# Make predictions on the test data
pipeline_test = full_pipeline.predict(X_test)

# Calculate accuracy score
accuracy = accuracy_score(y_true=y_test, y_pred=pipeline_test)
accuracy

0.9486301369863014

## 4. Train our model with GridSearch cross validation

### 4.1 Find out more about the current parameters

In [ ]:
full_pipeline.get_params()

### 4.2 Defining ParameterGrid

In [ ]:
paramGrid = {
    'preprocessor__num_pipeline__imputer__strategy': ['mean', 'median'],
    'preprocessor__cat_pipeline__imputer__strategy': ['most_frequent', 'constant'],
    'preprocessor__cat_pipeline__ordinal_encoder__handle_unknown': ['use_encoded_value'],
    'preprocessor__cat_pipeline__ordinal_encoder__unknown_value': [-1],
    'preprocessor__cat_pipeline__onehot_encoder__handle_unknown': ['ignore'],
    'preprocessor__cat_pipeline__onehot_encoder__sparse': [False, True],
    'randomforestclassifier__n_estimators': [100, 200, 300],
    'randomforestclassifier__max_depth': [None, 5, 10],
    'randomforestclassifier__min_samples_split': [2, 5, 10],
    'randomforestclassifier__min_samples_leaf': [1, 2, 4],
    'randomforestclassifier__max_features': ['auto', 'sqrt']
}


### 4.3 Define cross validation: GridSearchCV

In [ ]:
# GridSearchCV for hyperparameter tuning

from sklearn.model_selection import GridSearchCV

Search = GridSearchCV(full_pipeline, paramGrid, cv=5, verbose=1)


In [ ]:
# Fit your "search" to the training data (X and y)
Search.fit(X_train, y_train)

### 4.4  Predict on the Training set

In [ ]:
# Predict on the training set using the best model found by GridSearchCV
train_pred = Search.predict(X_train)

# Calculate accuracy score on the training set
train_accuracy = accuracy_score(y_true=y_train, y_pred=train_pred)
train_accuracy

0.9948630136986302

### 4.5  Predict on the Test set

In [ ]:
# Predict on the test set using the best model found by GridSearchCV
test_pred = Search.predict(X_test)

# Calculate accuracy score on the test set
test_accuracy = accuracy_score(y_true=y_test, y_pred=test_pred)

# Print the accuracy for the testing data set
print("Accuracy for the testing data set is:", test_accuracy)


Accuracy for the testing data set is: 0.9486301369863014


## 5. Competitions database

In [ ]:
# Step 1: Extract Sheet ID and Sheet name from the link
link = "https://docs.google.com/spreadsheets/d/13ktHYMi3WqMnR3YESS21tDeRSG3UnLZYqmHhLsNC_ho/edit#gid=2025370364"

sheet_id = link.split("/")[5]
sheet_name = link.split("gid=")[1]

# Step 2: Construct the URL
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={sheet_name}"

# Step 3: Read the URL as a pandas DataFrame
new_data  = pd.read_csv(url)

# Print the DataFrame
new_data

,Id,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,1461,11622,80.0,882.0,2,0,0,1.0,140,120,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1,1462,14267,81.0,1329.0,3,0,0,1.0,393,0,...,Attchd,Unf,TA,TA,Y,NaN,NaN,Gar2,WD,Normal
2,1463,13830,74.0,928.0,3,1,0,2.0,212,0,...,Attchd,Fin,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
3,1464,9978,78.0,926.0,3,1,0,2.0,360,0,...,Attchd,Fin,TA,TA,Y,NaN,NaN,NaN,WD,Normal
4,1465,5005,43.0,1280.0,2,0,0,2.0,0,144,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,1936,21.0,546.0,3,0,0,0.0,0,0,...,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,WD,Normal
1455,2916,1894,21.0,546.0,3,0,0,1.0,0,0,...,CarPort,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
1456,2917,20000,160.0,1224.0,4,1,0,2.0,474,0,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
1457,2918,10441,62.0,912.0,3,0,0,0.0,80,0,...,NaN,NaN,NaN,NaN,Y,NaN,MnPrv,Shed,WD,Normal


In [ ]:
# Removing irrelevant columns:

# Define the list of columns to be removed
columns_to_remove = ['Alley', 'Utilities', 'PoolQC', 'Fence', 'MiscFeature']

# Remove the columns from the dataframe
new_data = new_data.drop(columns_to_remove, axis=1)

In [ ]:
comp_pred = Search.predict(new_data)

In [ ]:
comp_pred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
comp_pred = pd.DataFrame(comp_pred)
comp_pred['Id']= new_data['Id']
comp_pred.rename(columns={0:'Expensive'}, inplace = True)
comp_pred = comp_pred[['Id', 'Expensive']]
comp_pred

,Id,Expensive
0,1461,0
1,1462,0
2,1463,0
3,1464,0
4,1465,0
...,...,...
1454,2915,0
1455,2916,0
1456,2917,0
1457,2918,0


In [ ]:
# Export the comp_pred DataFrame to a CSV file
comp_pred.to_csv("comp_pred.csv", index = False)

In [ ]:
# Download the CSV file
from google.colab import files
files.download("comp_pred.csv")